In [6]:
#function check เรียนไปแล้วหรือยัง (ตาราง subject)
import sqlite3
def check_already_regis(sub_id,u_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    #เลือกวิชาทั้งหมดที่มีจาก select_subjects ของ user คนนั้น
    c.execute("SELECT sub_id_id FROM app_schedule_user_subjects WHERE user_id_id = ?", (u_id,))
    user_subject = c.fetchall()

    # Close the connections
    conn.close()

    if (sub_id,) in user_subject :
        return False
    return True


In [7]:
#function check ลงไปแล้วรึยัง (ตารางเก็บวิชาที่ผู้ใช้เลือก)
import sqlite3
def check_pass_subject(sub_id,u_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    #เลือกวิชาทั้งหมดที่มีจาก subjects ของ user คนนั้น
    c.execute("SELECT real_subject_id FROM subjects WHERE UserID = ?", (u_id,))
    total_subjects = c.fetchall()

    #ไปค้นหารหัสวิชาจาก sub_id ที่ user ป้อนเข้ามา
    c.execute("SELECT code FROM app_schedule_subjects_info WHERE ID = ?", (sub_id,))
    select_sub = c.fetchall()

    # Close the connections
    conn.close()

    if select_sub[0] in total_subjects:
        return False
    else:
        return True

In [8]:
#Function check credit
import sqlite3
def check_over_credit(user_id,sub_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    # Execute the SELECT statement and retrieve the total sum of credits
    c.execute("SELECT SUM(app_schedule_subjects_info.credit) FROM app_schedule_subjects_info,app_schedule_user_subjects "+
              "WHERE app_schedule_user_subjects.user_id_id = ?  AND app_schedule_user_subjects.sub_id_id = app_schedule_subjects_info.ID ",(user_id,))
    result = c.fetchone()

    # Get the total sum of credits from the result tuple
    total_credits = result[0]
 

    # Execute the SELECT statement to retrieve the sum of credits for the specified subject
    c.execute("SELECT credit FROM app_select_subjects_test_date WHERE code = ?", (sub_id,))
    result = c.fetchone()

    # Get the sum of credits for the specified subject from the result tuple
    subject_credits = result[0]

    # Close the connections
    conn.close()
    
    credits_now = total_credits + subject_credits
    maxcredit = 22
    
    if credits_now > maxcredit :
        return False
    else:
        return True


In [1]:
#function check time

def Check_time_Overlapse(starttime_1,endtime_1,starttime_2,endtime_2):
    hour_starttime_1 = int(starttime_1.split(':')[0])
    min_starttime_1 = int(starttime_1.split(':')[1])

    hour_endtime_1 = int(endtime_1.split(':')[0])
    min_endtime_1 = int(endtime_1.split(':')[1])

    hour_starttime_2 = int(starttime_2.split(':')[0])
    min_starttime_2 = int(starttime_2.split(':')[1])

    hour_endtime_2 = int(endtime_2.split(':')[0])
    min_endtime_2 =  int(endtime_2.split(':')[1])

    

    if hour_endtime_1 <= hour_starttime_2 or  hour_endtime_2 <=  hour_starttime_1:              #เช็คว่ามีชั่วโมงซ้อนกันหรือไม่
        if hour_starttime_1 == hour_starttime_2 :                                                   #เวลาเริ่มเท่ากันน
            if min_starttime_1 == min_starttime_2:                                                      #นาทีเริ่มเท่ากันไม่ได้
                return False
            elif  min_starttime_1 < min_starttime_2:                                                    #นาทีตอนเริ่มอันแรกน้อยกว่าอันสอง หมายความว่าอันแรกเกิดก่อน
                if hour_endtime_1 > hour_starttime_2:                                                       #ชั่วโมงจบของอันแรกก็จะมากกว่าชั่วโมงเริ่มอันที่2ไม่ได้
                    return False
                elif hour_endtime_1 == hour_starttime_2:                                                    #ถ้าชั่วโมงจบของอันแรกเท่ากับชั่วโมงเริ่มอันที่2ต้องมาเช็คนาทีต่อ
                    if min_endtime_1 <= min_starttime_2:                                                        #ถ้านาทีจบของอันแรกเท่าหรือน้อยกว่านาทีเริ่มอันที่2ก็ไม่เป็นไร เพราะเท่ากับว่าอันแรกจบแล้วถึงมาเริ่มอันสอง
                        return True
                    else:
                        return False                                                                            #นาทีมากกว่าไม่ได้เพราไม่งั้นเท่ากับว่ามันจะจบหลังจากอันสองเริ่ม
            elif  min_starttime_1 > min_starttime_2:                                                    #นาทีตอนเริ่มอันแรกมากกว่าอันสอง หมายความว่าอันสองเกิดก่อน
                if hour_endtime_2 > hour_starttime_1:                                                       #ชั่วโมงจบของอันสองมากกว่าชั่วโมงเริ่มอันแรกไม่ได้
                    return False
                elif hour_endtime_2 == hour_starttime_1:                                                    #ถ้าชั่วโมงจบของอันสองเท่ากับชั่วโมงเริ่มอันแรกต้องมาเช็คนาทีต่อ
                    if min_endtime_2 <= min_starttime_1:                                                        #ถ้านาทีจบของอันสองเท่าหรือน้อยกว่าก็ไม่เป็นไร
                        return True
                    else:
                        return False                                                                            #นาทีมากกว่าไม่ได้
        elif hour_starttime_1 == hour_starttime_2 and  hour_endtime_2 ==  hour_endtime_1:           #ทั้งเวลาเริ่มและเวลาจบเท่ากันไม่ได้
            return False       
        else:
            return True                                                                         #ชั่วโมงไม่ได้มีเท่ากันเลยก็ไม่เป็นไร
    else:
        return False                                                                            #ชั่วโมงซ้อนทับกัน
    
    

False
False
False
True
True
False
False
False
False
False
False
True
True
False


In [7]:
#function check วันเวลาเรียน
import sqlite3
from datetime import datetime

def check_study_day(sub_id,user_id):
      conn = sqlite3.connect("w3.db")
      c = conn.cursor()
      c.execute("SELECT sub_id_id FROM app_schedule_user_subjects WHERE user_id_id = ? ",(user_id,)) 
      totalsubject = c.fetchall() #subject ทั้งหมด ที่ user ได้ select ไปแล้ว

      totaltimeofsub = [] #เก็บ day,start time,end time ของแต่ละ subject ที่ user select ไปแล้ว
      for x in totalsubject:
            sub = x[0]
            c.execute("SELECT day,start_time,end_time FROM app_schedule_subjects_info WHERE ID = ? ",(sub,))
            time = c.fetchall()
            totaltimeofsub.append(time[0])

      c.execute("SELECT day,start_time,end_time FROM app_schedule_subjects_info WHERE ID = ? ",(sub_id,))
      subject_select = c.fetchall() #เก็บ  day,start time,end time ของวิชาที่ user ต้องการ select ตอนนี้

      for y in totaltimeofsub:
            print(y[0],y[1],y[2])
            day_subject_selected = y[0]         # day จาก subject ที่ user ได้ select ไปแล้ว
            starttime_subject_selected = y[1]   # start time จาก subject ที่ user ได้ select ไปแล้ว
            endtime_subject_selected = y[2]     # end time จาก subject ที่ user ได้ select ไปแล้ว

            day_subject_select = subject_select[0][0]           # dayจาก subject ที่ user ต้องการ select 
            starttime_subject_select = subject_select[0][1]     # start time จาก subject ที่ user ต้องการ select 
            endtime_subject_select = subject_select[0][2]       # end time จาก subject ที่ user ต้องการ select 


            if day_subject_select == day_subject_selected:
                  if Check_time_Overlapse(starttime_subject_selected,endtime_subject_selected,starttime_subject_select,endtime_subject_select):
                        return True
                  else: 
                        return False
      return True
            
   


check_study_day("433",1)

[(430,), (431,), (432,), (433,)]
[('S', '09:00:00', '12:00:00')]
M 13:00:00 16:00:00
H 09:00:00 12:00:00
H 13:00:00 16:00:00
S 09:00:00 12:00:00


False

In [9]:
#function check วันเวลาสอบกลางภาค
from datetime import date
def check_midterm_day(sub_id,user_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    c.execute("SELECT testdate.mid_numday ,testdate.mid_starttime,testdate.mid_endtime "+
              "FROM app_schedule_user_subjects AS user , app_schedule_subjects_info AS info, app_select_subjects_test_date AS testdate " +
              "WHERE user.user_id_id = ?"+
               "AND user.sub_id_id = info.ID " +
               "AND info.code = testdate.code ",(user_id,))  
    totalsubject = c.fetchall()

    c.execute("SELECT testdate.mid_numday ,testdate.mid_starttime,testdate.mid_endtime "+
              "FROM app_schedule_subjects_info AS info, app_select_subjects_test_date AS testdate " +
              "WHERE ?= info.ID " +
               "AND info.code = testdate.code ",(sub_id,))  
    subject_select = c.fetchall()

    for x  in totalsubject:
        sub_selected = x[0].split("-")
        year,month,day = int(sub_selected[0]),int(sub_selected[1]),int(sub_selected[2])

        day_subject_selected = date(year,month,day)                                     # day จาก subject ที่ user ได้ select ไปแล้ว
        starttime_subject_selected = x[1]                                               # start time จาก subject ที่ user ได้ select ไปแล้ว
        endtime_subject_selected = x[2]                                                 # end time จาก subject ที่ user ได้ select ไปแล้ว

        sub_select = subject_select[0][0].split("-")
        year,month,day = int(sub_select[0]),int(sub_select[1]),int(sub_select[2])

        day_subject_select = date(year,month,day)                                       # dayจาก subject ที่ user ต้องการ select 
        starttime_subject_select = subject_select[0][1]                                 # start time จาก subject ที่ user ต้องการ select 
        endtime_subject_select = subject_select[0][2]                                   # end time จาก subject ที่ user ต้องการ select

        if day_subject_select == day_subject_selected:
            if Check_time_Overlapse(starttime_subject_selected,endtime_subject_selected,starttime_subject_select,endtime_subject_select):
                return True
            else: 
                return False
    return True
        

check_midterm_day(592,1)

True

In [42]:
#function check วันเวลาปลายภาค
from datetime import date
def check_final_day(sub_id,user_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    c.execute("SELECT testdate.fin_numday ,testdate.fin_starttime,testdate.fin_endtime "+
              "FROM app_schedule_user_subjects AS user , app_schedule_subjects_info AS info, app_select_subjects_test_date AS testdate " +
              "WHERE user.user_id_id = ?"+
               "AND user.sub_id_id = info.ID " +
               "AND info.code = testdate.code ",(user_id,))  
    totalsubject = c.fetchall()
    print(totalsubject)

    c.execute("SELECT testdate.fin_numday ,testdate.fin_starttime,testdate.fin_endtime "+
              "FROM app_schedule_subjects_info AS info, app_select_subjects_test_date AS testdate " +
              "WHERE ?= info.ID " +
               "AND info.code = testdate.code ",(sub_id,))  
    subject_select = c.fetchall()
    #print(subject_select)


    for x  in totalsubject:
        sub_selected = x[0].split("-")
        year,month,day = int(sub_selected[0]),int(sub_selected[1]),int(sub_selected[2])
        #print(year,month,day)
        day_subject_selected = date(year,month,day)                                     # day จาก subject ที่ user ได้ select ไปแล้ว
        starttime_subject_selected = x[1]                                               # start time จาก subject ที่ user ได้ select ไปแล้ว
        endtime_subject_selected = x[2]                                                 # end time จาก subject ที่ user ได้ select ไปแล้ว
        #print(day_subject_selected,starttime_subject_selected,endtime_subject_selected)

        sub_select = subject_select[0][0].split("-")
        year,month,day = int(sub_select[0]),int(sub_select[1]),int(sub_select[2])
        day_subject_select = date(year,month,day)                                       # dayจาก subject ที่ user ต้องการ select 
        starttime_subject_select = subject_select[0][1]                                 # start time จาก subject ที่ user ต้องการ select 
        endtime_subject_select = subject_select[0][2]                                   # end time จาก subject ที่ user ต้องการ select
        print(day_subject_select,starttime_subject_select,endtime_subject_select)

        if day_subject_select == day_subject_selected:
            if Check_time_Overlapse(starttime_subject_selected,endtime_subject_selected,starttime_subject_select,endtime_subject_select):
                return True
            else: 
                return False
    return True
        

check_final_day(592,1)

[('2566-03-31', '09:00:00', '12:00:00'), ('2566-03-27', '09:00:00', '12:00:00'), ('2566-03-28', '09:00:00', '12:00:00')]
2566-04-07 09:00:00 12:00:00
2566-04-07 09:00:00 12:00:00
2566-04-07 09:00:00 12:00:00


True

===Not use===

In [6]:
# add 0 front of subject_code
import sqlite3
user_id = 8
conn = sqlite3.connect("w3.db")
c = conn.cursor()
c.execute("SELECT * FROM subjects WHERE UserID = ?",(user_id,))
result = c.fetchall()
for a in result:
    x = '0'+a[1]
    print(x)
    c.execute("UPDATE subjects SET real_subject_id = ? WHERE UserID = ? AND real_subject_id = ? ",(x,user_id,a[1],))

conn.commit()
conn.close()

010113030
010123102
040203111
040313005
040313006
080103001
080303502
080303601
010113138
010113139
010123103
040003004
040203112
040313007
040313008
080103002
010123105
010123106
010123107
010123108
010123131
010123133
080103034


In [18]:
from datetime import datetime

event1_start = datetime(2023, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2023, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [19]:
from datetime import datetime

event1_start = datetime(2566, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2566, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [20]:
#function check วันเวลา
def check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    if event1_end <= event2_start or event2_end <= event1_start:
        return False
    else:
        return True

if check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    print("The two events overlap.")
else:
    print("The two events do not overlap.")


The two events do not overlap.


In [23]:
def Check_time_Overlapse(start_time_1, end_time_1, start_time_2, end_time_2):
    # Convert the time strings to datetime objects
    start_1 = datetime.strptime(start_time_1, '%H:%M:%S')
    end_1 = datetime.strptime(end_time_1, '%H:%M:%S')
    start_2 = datetime.strptime(start_time_2, '%H:%M:%S')
    end_2 = datetime.strptime(end_time_2, '%H:%M:%S')
    
    # Check for collision
    if start_1 >= end_2 and start_2 >= end_1:
        return True
    else:
        return False


In [28]:
#Check_time_Overlapse("9:30:00","11:20:00","9:45:00","15:50:00")
Check_time_Overlapse("13:00:00","13:30:00","13:50:00","14:00:00")

False